In [15]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import (
    FewShotPromptTemplate,
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate
)
import pickle
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
import bitsandbytes as bnb
def get_embedding():
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
        )
    return embeddings

def make_dict(dir='train.csv'):
    df = pd.read_csv(dir)
    df.drop('SAMPLE_ID', axis=1, inplace=True)
    
    return df.to_dict(orient='records')



def get_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
        )
    return embeddings

def load_and_vectorize(csv_path, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    # CSV 파일을 불러와 데이터프레임 생성
    train_df = pd.read_csv(csv_path)
    train_df.drop('SAMPLE_ID', axis=1, inplace=True)
    trainset = train_df.to_dict(orient='records')
    print("Loaded Fewshot Set:", trainset[:3])
    
    # 벡터화할 텍스트 생성
    to_vectorize = ["\n\n".join(example.values()) for example in trainset]
    
    # 벡터화 및 FAISS DB 생성
    fewshow_vectordb = FAISS.from_texts(to_vectorize, embedding=get_embedding(), metadatas=trainset)
    
    # FAISS DB 저장
    save_faiss_db(fewshow_vectordb, db_path)
    
    return fewshow_vectordb

def save_faiss_db(db, db_path):
    db.save_local(db_path)
    with open(db_path + '_metadata.pkl', 'wb') as f:
        pickle.dump(db, f)

def load_faiss_db(db_path):
    db = FAISS.load_local(db_path, embeddings=get_embedding(), allow_dangerous_deserialization=True)
    with open(db_path + '_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    return db, metadata

def load_chunks_make_docdb(pdf_directory, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    print("Loading PDF files from:", pdf_directory)
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf').replace('\\', '/'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)
    
    # 분할된 텍스트를 벡터로 변환하여 FAISS DB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    print("Done.", len(chunks), "chunks")
    
    print("Creating FAISS DB")
    # FAISS DB 생성 및 저장
    db = FAISS.from_documents(chunks, embedding=get_embedding())
    save_faiss_db(db, db_path)
    print("Done.")
    
    return db


In [ ]:
def make_fewshot_prompt(fewshot_vectordb, k = 3):
    # Semantic Similarity Example Selector 설정
    example_prompt = PromptTemplate.from_template("Question: {Question}\nAnswer: {Answer}\nSource: {Source}")

    example_selector = SemanticSimilarityExampleSelector(
        vectorstore=fewshot_vectordb,
        k=k,
    )
    # FewShotPromptTemplate 생성
    fewshot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        suffix="Question: {input}",
        input_variables=["input"],
    )
    return fewshot_prompt

def make_fewshot_string(fewshot_prompt, train_retriever, buff):
    ex_qa = fewshot_prompt.invoke({"input": buff['Question']}).to_string()
    fewshot_list = ex_qa.split('\n\n')[:-1]
    for i, entry in enumerate(fewshot_list):
        question = entry.split('\n')[0]
        question = question.replace('Question: ', '')
        retrieved_docs = train_retriever.invoke(question)
        num = "Example {}\n".format(i+1)
        fewshot_list[i] = num + retrieved_docs[0].page_content + entry + '\n\n'
    return str(fewshot_list)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [3]:
def setup_llm_pipeline(model_id):
    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False
    
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
    )
    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        #trust_remote_code=True 
        )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        #model_kwargs={"torch_dtype": torch.bfloat16},
        eos_token_id=[
                128001,
                128008,
                128009
        ]
        ,
        bos_token_id = 128000,

        do_sample = True,
        temperature=0.6,
        top_p=0.9,
        return_full_text=False,
        # eos_token_id=terminators,
        max_new_tokens=128,
        #pad_token_id=tokenizer.eos_token_id  # 패딩 토큰을 EOS 토큰 ID로 설정
    )


    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm

In [4]:
fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')
fewshot_prompt = make_fewshot_prompt(fewshot_db)


Loading FAISS DB from: ./fewshot_faiss_db


NameError: name 'make_fewshot_prompt' is not defined

In [5]:

train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
train_retriever = train_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})

test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
test_retriever = test_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})

train_dict = make_dict('train.csv')
test_dict = make_dict('test.csv')

Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db


In [4]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

llm = setup_llm_pipeline(model_id)

Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.86s/it]
Some parameters are on the meta device device because they were offloaded to the disk and cpu.


In [16]:

i = 1
#fewshot_str = make_fewshot_string(fewshot_prompt, train_retriever, test_dict[i])
        
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=fewshot_db,
    k=3,
)
# The prompt template will load examples by passing the input do the `select_examples` method
fewshot_list = example_selector.select_examples({"input": test_dict[i]['Question']})
for i, entry in enumerate(fewshot_list):
        question = entry['Question']
        retrieved_docs = train_retriever.invoke(question)
        fewshot_list[i]['retrieved_docs'] = retrieved_docs
print(fewshot_list)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)


[{'Source': '중소벤처기업부_창업사업화지원', 'Source_path': './train_source/중소벤처기업부_창업사업화지원.pdf', 'Question': '창업사업화지원의 사업목적은 무엇인가?', 'Answer': '창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.', 'retrieved_docs': [Document(metadata={'Source': '중소벤처기업부_창업사업화지원', 'Source_path': './train_source/중소벤처기업부_창업사업화지원.pdf', 'Question': '창업사업화지원의 사업목적은 무엇인가?', 'Answer': '창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.'}, page_content='중소벤처기업부_창업사업화지원\n\n./train_source/중소벤처기업부_창업사업화지원.pdf\n\n창업사업화지원의 사업목적은 무엇인가?\n\n창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.')]}, {'Source': '중소벤처기업부_창업사업화지원', 'Source_path': './train_source/중소벤처기업부_창업사업화지원.pdf', 'Question': "언제 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 '기술혁신형 창업기업 지원사업'이 신설되었나요?", 'Answer': "기술혁신형 창업기업 지원사업(현, 예비창업패키지)'은 '18년에 신설되었습니다.", 'retrieved_docs': [Document(metadata={'Source': '중소벤처기업부_창업사업화지원', 'Source_path': './

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{context}\n\n{input}"),
    ]
)

NameError: name 'few_shot_prompt' is not defined

In [6]:
qa_chain = (
        {
            "context": test_retriever | format_docs,
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

answer = qa_chain.invoke(test_dict[i]['Question'])

In [7]:
print(answer)   

중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 창업기반지원과 개발기술사업화를 통해 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의 창업을 활성화하고 고


In [ ]:
results =[]
for i in tqdm(range(len(test_dict))):
    print(test_dict[i]['Question'])
    fewshot_str = make_fewshot_string(fewshot_prompt, train_retriever, test_dict[i])
    # print(fewshot_str)
    full_template = """
너는 사람들에게 방대한 재정 데이터를 일반 국민과 전문가들이 이해할 수 있는 방식으로 전달하고 싶어한다.
너는 이를 위한 몇가지 예시 참고해서  재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 응답을 제시해줘.
반드시 context를 참고하여 답변을 작성해야해.



"""+f"""
이건 질문과 관련된 예시야.

{fewshot_str}
"""+"""

이제 진짜로 한번 해봐. 토큰은 64개 이내로 작성해야해.
Context와 위의 예시를 참고하여 아래 질문에 대한 답변만 작성해. 한 문장으로 작성해줘.
Context:{context}

Question: {input}


"""
    prompt = PromptTemplate.from_template(full_template)
    qa_chain = (
    {
        "context": test_retriver | format_docs,
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    )
    answer = qa_chain.invoke(test_dict[i]['Question'])
    results.append({
        "Question": test_dict[i]['Question'],
        "Answer": qa_chain.invoke(test_dict[i]['Question']),
        "Source": test_dict[i]['Source']
    
        })
    print("================================================")
    print("Questions: ",results[-1]['Question'])
    print("Answer: ",results[-1]['Answer'])
    #print(results[-1]['Source'])
        
    # 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")
# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)

In [ ]:

def setup_llm_pipeline(model_id):
    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False
    # 패딩 토큰을 EOS 토큰으로 설정
    tokenizer.pad_token = tokenizer.eos_token
    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        #trust_remote_code=True 
        )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        model_kwargs={"torch_dtype": torch.bfloat16},
        temperature=0.1,
        return_full_text=False,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id  # 패딩 토큰을 EOS 토큰 ID로 설정
    )


    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm